In [1]:
from dotenv import load_dotenv
_ =load_dotenv(override=True)

2.1 Initialize the agent's state

In [2]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [3]:
from typing import Literal , Optional, List, Dict, Any ,Type
from langgraph.graph import MessagesState

#Custom State class with specific keys
class State(MessagesState):
    user_query: Optional[str]  # The user's original query
    enabled_agents: Optional[List[str]]  # Makes our multi-agent system modular on which agents to include
    plan: Optional[List[Dict[int, Dict[str,Any]]]] # Listing the steps in the plan needed to achieve the goal.
    current_step: int #Marking the current step in the plan.
    agent_query: Optional[str] # Inbox note: `agent_query` tells the next agent exactly what to do at the current step.
    last_reason: Optional[str] # Explains the exceutor's decision to help maintain continuity and provide traceability.
    replan_flag: Optional[bool] # Set by the executor to indicate that the planner should revise the plan.
    replan_attempts: Optional[Dict[int,Dict[int,int]]] # Replan attempts tracked per step number.

In [4]:
pip install langchain_openai

  Using cached openai-2.1.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.1.0-py3-none-any.whl (964 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.99.9
    Uninstalling openai-1.99.9:
      Successfully uninstalled openai-1.99.9
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trulens-providers-openai 2.4.0 requires openai<1.100.0,>=1.52.1, but you have openai 2.1.0 which is incompatible.


In [5]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [6]:
from prompts import plan_prompt
from langgraph.types import Command
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI
import json

In [7]:
reasoning_llm = ChatOpenAI(
    model= "o3",
    model_kwargs= {" response_format": {"type":"json_objet"}},
)

2.2 Create planner.

In [8]:
def planner_node(state: State) -> Command[Literal['executor']]:
    """
    Runs the planning LLM and stores the resulting plan in state.
    """
    # 1. Invoke LLM with the planner prompt
    llm_reply = reasoning_llm.invoke([plan_prompt(state)])

    # 2. Validate JSON
    try:
        content_str = llm_reply.content if isinstance( llm_reply.content, str)else str(llm_reply.content)
        parsed_plan= json.loads(content_str)
    except json.JSONDecodeError:
        raise ValueError(
            f"Planner returned invalid JSON:\n{llm_reply.content}")
    # 3. Store as current plan only
    replan = state.get("replan_flag", False)
    updated_plan: Dict[str, Any] = parsed_plan

    return Command(
        update={
            "plan" : updated_plan,
            "messages": [HumanMessage(
                content=llm_reply.content,
                name="replan" if replan else "initial_plan")],
            "user_query": state.get("user_query", state["messages"][0].content),
                                    "current_step": 1 if not replan else state["current_step"],
            # Preserve replan flag so executor runs planned agent
            # once before reconsidering
            "replan_flag": state.get("replan_flag", False),
            "last_reason":"",
            "enabled_agents": state.get("enabled_agents"),
        },
        goto="executor",
    )

2.3 Create executor

In [9]:
from prompts import executor_prompt
from langgraph.graph import END

MAX_REPLANS=3

In [10]:
def executor_node(
    state: State,
) -> Command[Literal["web_researcher", "chart_generator", "synthesizer","planner"]]:
    plan: Dict[str, Any] = state.get("plan",{})
    step: int = state.get("current_step",1)
    # 0) If we *just* replanned, 
    # run the planned agent once before reconsidering.
    if state.get("replan_flag"):
        planned_agent=plan.get(str(step),{}).get("agent")
        return Command(
            update={
                "replan_flag":False,
                "current_step": step+1, # advance because we executed the planned agent
            },
            goto=planned_agent,
        )
    # 1) Build prompt & call LLM
    llm_reply = reasoning_llm.invoke([executor_prompt(state)])
    try:
        content_str = llm_reply.content if isinstance(llm_reply.content,str)else str(llm_reply.content)
        parsed=json.loads(content_str)
        replan: bool =parsed["replan"]
        goto: str = parsed["goto"]
        reason: str = parsed["reason"]
        query: str =parsed["query"]
    except Exception as e:
        raise ValueError(f" Invalid executor JSON: \n{llm_reply.content}") from e

    # Update the state
    updates: Dict[str, Any] ={
    "messages": [HumanMessage(content=llm_reply.content, name="executor")],
    "last_reason": reason,
    "agent_query": query,
    }

    # Replan accounting
    replans: Dict[int,int] =state.get("replan_attempts",{}) or {}
    step_replans = replans.get(step,0)

    # 2) Replan decision
    if replan:
        if step_replans < MAX_REPLANS:
            replans[step] = step_replans +1
            updates.update({
                "replan_attempts": replans,
                "replan_flag": True,  # ensure next turn executes the planned agent once
                "current_step" :step, # stay on same step for the new plan
            })
            return Command(update=updates, goto="planner")
        else:
              # Cap hit: skip this step; let next step (or synthesizer) handle termination
            next_agent= plan.get(str(step+1),{}).get("agent", "synthesizer")
            updates["current_step"] = step + 1
            return Command(update= updates, goto= next_agent)
    # 3) Happy path: run chosen agent; advance only if following the plan
    planned_agent = plan.get (str(step),{}).get("agent")
    updates["current_step"] = (step + 1) if goto == planned_agent else step
    updates["replan_flag"] = False
    return Command(update= updates, goto=goto)

2.4 Create Web research agent

In [11]:
pip install langchain_tavily

Note: you may need to restart the kernel to use updated packages.


In [12]:
from langgraph.prebuilt import create_react_agent
from typing import Literal
from langchain_tavily import TavilySearch
from langchain_openai import ChatOpenAI
tavily_tool = TavilySearch(max_results=5)
tavily_tool.invoke(" What is NVIDIA 's stock price?")['results']

[{'url': 'https://tradingeconomics.com/nvda:us',
  'title': 'Nvidia | NVDA - Stock Price | Live Quote | Historical Chart',
  'content': 'Nvidia Corporation traded at $187.62 this Friday October 3rd, decreasing $1.27 or 0.67 percent since the previous trading session. Looking back, over the',
  'score': 0.8941352,
  'raw_content': None},
 {'url': 'https://www.marketwatch.com/investing/stock/nvda?gaa_at=eafs&gaa_n=ASWzDAgZUNEzM8fMRnlgzIprQCZ-hnTYzeKNO0dxMFlbTQrLHgHarZyyr3FW&gaa_ts=68e30d1b&gaa_sig=U2i56HyLEW9_Z8m1vcBdNi-RWtYyzA4bisd4OAxeFPFyBlwxF9yZgjLO0xdpNAINK1S2P7oJs8vr7IHZjqh14Q%3D%3D',
  'title': 'NVIDIA Corp. Stock Quote (U.S.: Nasdaq) - NVDA - MarketWatch',
  'content': 'NVIDIA Corp. ; Open $189.19 ; Day Range 185.38 - 190.36 ; 52 Week Range 86.62 - 191.05 ; Market Cap $4.56T ; Public Float 23.31B',
  'score': 0.8846886,
  'raw_content': None},
 {'url': 'https://www.tradingview.com/symbols/NASDAQ-NVDA/',
  'title': 'NVIDIA Stock Chart — NASDAQ:NVDA Stock Price - TradingView',
  'c

In [13]:
pip install trulens-providers-openai

  Using cached openai-1.99.9-py3-none-any.whl.metadata (29 kB)
Using cached openai-1.99.9-py3-none-any.whl (786 kB)
  Attempting uninstall: openai
    Found existing installation: openai 2.1.0
    Uninstalling openai-2.1.0:
      Successfully uninstalled openai-2.1.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.34 requires openai<3.0.0,>=1.104.2, but you have openai 1.99.9 which is incompatible.


In [14]:
pip install langchain_experimental

In [15]:
from helper import agent_system_prompt

llm= ChatOpenAI(model="gpt-4o")

#Research agent and node
web_search_agent= create_react_agent(
    llm,
    tools= [tavily_tool],
    prompt=agent_system_prompt(f"""
          You are the Researcher. You can ONLY perform research 
        by using the provided search tool (tavily_tool). 
        When you have found the necessary information, end your output.  
        Do NOT attempt to take further actions.
        """),
)

In [16]:
agent_response= web_search_agent.invoke(
    {"messages":" what is NVIDIA's current market cap?"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
agent_response['messages'][-1].content

In [ ]:
def web_research_node(
    state: State,
) -> Command[Literal["executor"]]:
    agent_query= state.get("agent_query")
    result= web_search_agent.invoke({"messages":agent_query})
    goto="executor"
      # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1]=HumanMessage(
    content=result["messages"][-1].content, name="web_researcher"
    )
    return Command(
    update={
        # share internal message history of research agent with other agents
        "messages": result["messages"],
    },
    goto=goto,
    )

2.5 Create charting agent.

In [ ]:
from helper import python_repl_tool
# Chart generator agent and node
# NOTE: THIS PERFORMS ARBITRARY CODE EXECUTION, 
# WHICH CAN BE UNSAFE WHEN NOT SANDBOXED
chart_agent = create_react_agent(
    llm,
    [python_repl_tool],
    prompt=agent_system_prompt(
        """
        You can only generate charts. You are working with a researcher 
        colleague.
        1) Print the chart first.
        2) Save the chart to a file in the current working directory.
        3) At the very end of your message, output EXACTLY two lines 
        so the summarizer can find them:
           CHART_PATH: <relative_path_to_chart_file>
           CHART_NOTES: <one concise sentence summarizing the main insight in the chart>
        Do not include any other trailing text after these two lines.
        """
    ),
)

In [ ]:
def chart_node(state: State) -> Command[Literal["chart_summarizer"]]:
    result = chart_agent.invoke(state)
    # wrap in a human message, as not all providers allow
    # AI message at the last position of the input messages list
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content, name="chart_generator"
    )
    goto="chart_summarizer"
    return Command(
        update={
            # share internal message history of chart agent with other agents
            "messages": result["messages"],
        },
        goto=goto,
    )

2.6 Create chart summary agent

In [ ]:
from langchain_community.tools import ImageCaptionTool
from langchain.tools import Tool

chart_summary_agent = create_react_agent(
    llm,
    tools=[
        ImageCaptionTool(), 
        Tool.from_function(extract_chart_text_tool, name="ExtractChartText"),
    ],
    prompt=agent_system_prompt(
        "You can only generate image captions. You are working with a researcher colleague and a chart generator colleague. "
        "Your task is to generate a concise summary for the provided chart image at a local PATH, using visual and text information."
    ),
)

In [ ]:
def chart_summary_node(
    state: State,
) -> Command[Literal[END]]:
    result = chart_summary_agent.invoke(state)
    print(f"Chart summarizer answer: {result['messages'][-1].content}")
    # Send to the end node
    goto = END
    return Command(
        update={
            # share internal message history of chart agent with other agents
            "messages": result["messages"],
            "final_answer": result["messages"][-1].content,
        },
        goto=goto,
    )

In [ ]:
2.7 Create a Synthesizer (Text Summarizer) Agent

In [ ]:
llm= ChatOpenAI(model="gpt-4o")

In [ ]:
def synthesizer_node(state: State) -> Command[Literal[END]]:
    """
    Creates a concise, human‑readable summary of the entire interaction,
    **purely in prose**.

    It ignores structured tables or chart IDs and instead rewrites the
    relevant agent messages (research results, chart commentary, etc.)
    into a short final answer.
    """
    # Gather informative messages for final synthesis
    relevant_msgs = [
        m.content for m in state.get("messages", [])
        if getattr(m, "name", None) in ("web_researcher", 
                                        "chart_generator", 
                                        "chart_summarizer")
    ]

    user_question = state.get("user_query", state.get("messages", [{}])[0].content if state.get("messages") else "")

    synthesis_instructions = (
        """
        You are the Synthesizer. Use the context below to directly 
        answer the user's question. Perform any lightweight calculations, 
        comparisons, or inferences required. Do not invent facts not 
        supported by the context. If data is missing, say what's missing
        and, if helpful, offer a clearly labeled best-effort estimate 
        with assumptions.\n\n
        Produce a concise response that fully answers the question, with 
        the following guidance:\n
        - Start with the direct answer (one short paragraph or a tight bullet list).\n
        - Include key figures from any 'Results:' tables (e.g., totals, top items).\n
        - If any message contains citations, include them as a brief 'Citations: [...]' line.\n
        - Keep the output crisp; avoid meta commentary or tool instructions.
        """
        )

    summary_prompt = [
        HumanMessage(content=(
            f"User question: {user_question}\n\n"
            f"{synthesis_instructions}\n\n"
            f"Context:\n\n" + "\n\n---\n\n".join(relevant_msgs)
        ))
    ]

    llm_reply = llm.invoke(summary_prompt)

    answer = llm_reply.content.strip()
    print(f"Synthesizer answer: {answer}")

    return Command(
        update={
            "final_answer": answer,
            "messages": [HumanMessage(content=answer, name="synthesizer")],
        },
        goto=END,           # hand off to the END node
    )

2.8 Build The agent graph

In [ ]:
from langgraph.graph import START, StateGraph

workflow= StateGraph(State)
workflow.add_node("planner",planner_node)
workflow.add_node("executor",executor_node)
workflow.add_node("web_researcher",web_research_node)
workflow.add_node("chart_generator",chart_node)
workflow.add_node("chart_summarizer",chart_summary_node)
workflow.add_node("synthesizer",synthesizer_node)

workflow.add_edge(START, "planner")
graph= workflow.compile()


In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_png()))

2.9 Use the agent

In [ ]:
from langchain.schema import HumanMessage
import json
query = "Chart the current market capitalization of the top 5 semiconductor industries in the US?"
print(f"Query: {query}")

state = {
            "messages": [HumanMessage(content=query)],
            "user_query": query,
            "enabled_agents": ["web_researcher", "chart_generator", 
                               "chart_summarizer", "synthesizer"],
        }
graph.invoke(state)

print("--------------------------------")

In [ ]:
query = "Identify current regulatory changes for the industrial services industry in the US."
print(f"Query: {query}")

state = {
            "messages": [HumanMessage(content=query)],
            "user_query": query,
            "enabled_agents": ["web_researcher", "chart_generator", 
                               "chart_summarizer", "synthesizer"],
        }
graph.invoke(state)

print("--------------------------------")